# BDSI Data Mining Group
# Day 3  - Example Modeling - 6/21/22

In [ ]:
library(tidyr)
library(dplyr)
library(stringr)
library(ggplot2)
library(repr)
library(maps)
library(randomForest)

In [ ]:
load("../temp/overdoseDatFull.Rdata")

In [ ]:
head(dat)

## Modeling Mortality

### Random Forest

In [ ]:
colnames(dat)

In [ ]:
mod.dat <- dat %>%
    filter(year > 2005) %>%
    mutate(year_cat = factor(year)) %>%
    mutate(fentynol = as.numeric(year >= 2013)) %>%
    select(-stateName, -stateFIPS, -year_cat, -mort, -stateAbr)

Random forest requires that you feed it data that has no missing values in the covariates or output. We can identify rows with complete cases (no missing values) using the `complete.cases` function. This returns a logical vecter the length of the data, which takes the value of `TRUE` if the row is not missing any data.

In [ ]:
cc <- complete.cases(mod.dat)

I'll fit a random forest model on all of the covariates first.

In [ ]:
mod.rf <- randomForest(rate ~ ., importance = T, data = mod.dat[cc,])

`print` gives me some information about the model fit - including that the R^2 of the model is almost 90%!

In [ ]:
print(mod.rf)

While random forest models don't provide covariates for variables, we can still look at the relative importance of different covariates in the model.

In [ ]:
mod.rf$importance

In [ ]:
imp = sort(mod.rf$importance[,2]/sum(mod.rf$importance[,2]),
           decreasing = T) %>%
    data.frame()

imp$var <- rownames(imp)
colnames(imp) <- c("imp", "var")
imp

In [ ]:
g <- ggplot(data=imp) + geom_col(aes(x=reorder(var,imp), y=imp), fill="#2C7BB6")
g + coord_flip() + xlab("Feature") + ylab("Variable Importance (Standardized)") +
  theme(plot.title = element_text(hjust = 0.5), panel.border = element_blank(),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

The  most important variable is year, followed by unemployrment rate and whether or not ilicit Fentanol was in the US.

I created a function so that I could see the general relationship between each variable and the outcome, holding all other variables constant.

In [ ]:
plot_var_funct_comb <- function(mod.obj, mod.var.names, plot.var, label, dat){
  
  #start the fake dataset to make predictions on a grid
  n.var <- which(names(dat) == plot.var)
  
  
  if(is.numeric(unlist(dat[,n.var]))){
    fake.dat <- data.frame(var = seq(min(dat[,n.var]), max(dat[,n.var]), length.out = 1000))
  }else{
    fake.dat <- data.frame(var = factor(levels(as.factor(unlist(dat[,n.var])))))
  }
  #add the median value for all other variables
  dat.vars = setdiff(mod.var.names, plot.var)
  
  for (var.name in dat.vars) {
    #get variable location
    n.var2 <- which(names(dat) == var.name)
    
    #if the variable is numeric, use the median
    if(is.numeric(unlist(dat[,n.var2]))){
      fake.dat <- fake.dat %>% 
      mutate(!!var.name := median(unlist(dat[,n.var2]), na.rm = T))
    }else{ 
    #if the variable is categorical, use the most occurring value
    fake.dat <- fake.dat %>% 
      mutate(!!var.name := factor(names(which(table(dat[,n.var2]) == max(table(dat[,n.var2]))))[1]))
    }
  }
  
  #fix the variable name for prediction
  colnames(fake.dat)[1] <- plot.var
  
  #predictions based on median values and 
  fake.dat$pred <- predict(mod.obj, type = "response", newdata = fake.dat)
  
  #fix the variable name for plotting
  colnames(fake.dat)[1] <- "var"
  
  #generate other plotting data with actual data values of variable and the predictions
  plot.dat <- data.frame(var = dat[,n.var], pred = predict(mod.obj, type = "response"))
  colnames(plot.dat)[1] <- "var"

  #plot
if(!is.numeric(unlist(dat[,n.var]))){
  g <- ggplot() +
    geom_boxplot(data = plot.dat,  aes(x = var, y = pred, group = var)) +
    geom_point(data = fake.dat, aes(x = var, y = pred) ,color = "blue", size = 1) +
    xlab(label) + ylab("Prediction") + 
    theme(plot.title = element_text(hjust = 0.5), 
            panel.border = element_blank(),
            panel.grid.minor = element_blank(),
           axis.text.x = element_text(angle=45, hjust = 1)
    )
    }else{
    g <- ggplot() +
    geom_point(data = plot.dat,  aes(x = var, y = pred)) +
    geom_point(data = fake.dat, aes(x = var, y = pred) ,color = "blue", size = 1) +
    xlab(label) + ylab("Prediction") + 
    theme(plot.title = element_text(hjust = 0.5), 
            panel.border = element_blank(),
            panel.grid.minor = element_blank(),
           axis.text.x = element_text(angle=45, hjust = 1))
    }
          
  g
  return(g)
  
}

In [ ]:
var.names <- rownames(mod.rf$importance)

for(var in var.names){
    p <- plot_var_funct_comb(mod.rf, var.names, var, var, mod.dat[cc,])
    print(p)
}

## GLM

We can also look at trends using a linear model to get a better idea of variable relationships and do inference. 

In [ ]:
mod.dat2 <- dat %>%
    filter(year > 2005) %>%
    mutate(year_cat = factor(year)) %>%
    mutate(across(pctUrban_2010:popDens_2010, scale))

For modeling mortality outcomes, I use a poisson regression model, with a population offset. This is standard in the field to best model mortality outcomes.

I am going to start by just looking and year and opioid dispensing.

In [ ]:
mod.simple <- glm(mort ~ year_cat + opioid_disp + opioid_disp*year_cat + 
                  offset(log(pop)), 
                  family = "poisson", data = mod.dat2)

In [ ]:
summary(mod.simple)

Now let's look at the model with all other factors included.

In [ ]:
mod.full <- glm(mort ~ year_cat + opioid_disp + opioid_disp*year_cat + 
                pctUrban_2010 + vetPop_2020 + mdcrEnr_2019 + mdcaidElig_2012 +
                medIncome_2019 + pctPovty_2019 + pctNoIns_2019 +
                unplmtRate_2020 + popDens_2010 +
                offset(log(pop)), 
                family = "poisson", data = mod.dat2)

In [ ]:
summary(mod.full)

Given that the data is grouped by state, it makes sense to try adding random state effects. To do this, we will use the `lme4` package.

In [ ]:
library(lme4)

In [ ]:
lmer.full <- glmer(mort ~ year_cat + opioid_disp + opioid_disp*year_cat + 
                pctUrban_2010 + vetPop_2020 + mdcrEnr_2019 + mdcaidElig_2012 +
                medIncome_2019 + pctPovty_2019 + pctNoIns_2019 +
                unplmtRate_2020 + popDens_2010 +
                offset(log(pop)) + (1|stateAbr), 
                family = "poisson", data = mod.dat2)

In [ ]:
summary(lmer.full)

### Visualize coefficients

Especially with a log-linear model and interactions, it can be hard to interpret raw coefficients. To communicate what we have found more clearly, we can create visualizations of the coefficient estimates.

In [ ]:
coef.df <- data.frame(summary(lmer.full)$coefficients)
coef.df$var = row.names(coef.df)
colnames(coef.df) <- c("est", "se", "t", "p","var")
coef.df <- coef.df %>%
  mutate(sig = ifelse(p < .05, "Yes", "No"),
         coef.exp = exp(est))

Look at the year and opioid dispersion interaction.

In [ ]:
coef.base <- coef.df$est[coef.df$var == "opioid_disp"]
disp.coef <- coef.df %>%
    filter(str_detect(var, "opioid")) %>%
    mutate(year = case_when(!str_detect(var, "\\d{4}")~ "2006",
                            TRUE ~ str_extract(var, "\\d{4}")),
          int.est = case_when(year == "2006" ~ est,
                              TRUE ~ est+coef.base),
          int.est.exp = exp(int.est),
          sig = case_when(year == "2006" ~ "NA",
                         TRUE ~ sig))

In [ ]:
g <- ggplot(data=disp.coef, aes(x=factor(year))) + 
    geom_bar(aes(y=int.est.exp-1, fill = sig), stat ="identity", position="dodge")
g1 <- g + xlab("Year") + ylab("Multiplicative Effect on Mortality - 1") +
  theme(plot.title = element_text(hjust = 0.5), panel.border = element_blank(),
        #panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        axis.text.x = element_text(angle=45, hjust = 1)) +
  scale_fill_manual(name = "Interaction\nsignificant?", values = c("grey","#fdae61", "#2C7BB6"))
g1